In [5]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import altair as alt
alt.renderers.enable('notebook')

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

/Users/marissakelley/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
data = pd.read_csv("filtered_groups_defg.csv")
print(data.shape)
data.head()

(54093, 40)


,Unnamed: 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,0,17,17,FS-1418878,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,...,G,38.523333,-120.211667,5.0,USFS,CA,5,5.0,Amador,b'\x00\x01\xad\x10\x00\x00dS\\\xf2\x8b\r^\xc0\...
1,1,18,18,FS-1418881,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,...,G,38.780000,-120.260000,5.0,USFS,CA,17,17.0,El Dorado,b'\x00\x01\xad\x10\x00\x00p=\n\xd7\xa3\x10^\xc...
2,2,40,40,FS-1418920,FED,FS-FIRESTAT,FS,USNCNCF,National Forests in North Carolina,0811,...,D,36.001667,-81.590000,14.0,MISSING/NOT SPECIFIED,NC,27,27.0,Caldwell,b'\x00\x01\xad\x10\x00\x00\xf4(\\\x8f\xc2eT\xc...
3,3,117,119,FS-1419153,FED,FS-FIRESTAT,FS,USNENBF,Nebraska National Forest,0207,...,D,43.899167,-102.954722,5.0,USFS,SD,103,103.0,Pennington,b'\x00\x01\xad\x10\x00\x00\x84\xea9+\x1a\xbdY\...
4,4,118,120,FS-1419156,FED,FS-FIRESTAT,FS,USNENBF,Nebraska National Forest,0207,...,D,43.892778,-102.948056,5.0,USFS,SD,103,103.0,Pennington,b'\x00\x01\xad\x10\x00\x00h?:\xf1\xac\xbcY\xc0...


In [8]:
print(list(data.columns))

['Unnamed: 0', 'OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME', 'Shape']


In [16]:
drop_cols = ['Unnamed: 0','OBJECTID', 'FOD_ID', 'FPA_ID','NWCG_REPORTING_UNIT_ID','NWCG_REPORTING_UNIT_NAME','SOURCE_REPORTING_UNIT','SOURCE_REPORTING_UNIT_NAME','LOCAL_FIRE_REPORT_ID','LOCAL_INCIDENT_ID','FIRE_CODE','FIRE_NAME','ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME','COMPLEX_NAME','DISCOVERY_DATE','STAT_CAUSE_CODE','FIPS_CODE', 'FIPS_NAME', 'Shape','OWNER_CODE','DISCOVERY_TIME','COUNTY','CONT_DATE','CONT_DOY','CONT_TIME','SOURCE_SYSTEM','SOURCE_SYSTEM_TYPE']
data_dropped = data.drop(columns = drop_cols)
data_dropped

,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE
0,FS,2004,280,Equipment Use,16823.00,G,38.523333,-120.211667,USFS,CA
1,FS,2004,287,Equipment Use,7700.00,G,38.780000,-120.260000,USFS,CA
2,FS,2005,43,Debris Burning,125.00,D,36.001667,-81.590000,MISSING/NOT SPECIFIED,NC
3,FS,2005,197,Lightning,119.00,D,43.899167,-102.954722,USFS,SD
4,FS,2005,197,Lightning,119.00,D,43.892778,-102.948056,USFS,SD
5,FS,2005,172,Lightning,16100.00,G,33.723333,-111.275000,USFS,AZ
6,FS,2005,190,Miscellaneous,1149.00,F,35.688056,-118.438333,USFS,CA
7,FS,2005,197,Lightning,1300.00,F,33.933611,-111.379722,USFS,AZ
8,FS,2005,198,Lightning,730.00,E,33.871111,-111.072500,USFS,AZ
9,FS,2005,198,Lightning,495.00,E,33.862500,-111.155833,USFS,AZ


In [17]:
data_dropped.columns

Index(['NWCG_REPORTING_AGENCY', 'FIRE_YEAR', 'DISCOVERY_DOY',
       'STAT_CAUSE_DESCR', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'OWNER_DESCR', 'STATE'],
      dtype='object')

In [22]:
#Y_train = data_dropped.iloc[0:, 0].values
#text_train = data_dropped.iloc[0:, 1].values

In [25]:
df_train = data_dropped.loc[data_dropped == 'train']
df_test = data_dropped.loc[data_dropped == 'test']

Y_train = df_train.iloc[0:, 2].values
text_train = df_train.iloc[0:, 3].values

Y_test = df_test.iloc[0:, 2].values
text_test = df_test.iloc[0:, 3].values

ValueError: Cannot index with multidimensional key

# Logistic Regression

In [ ]:
acc_train = []
acc_test = []

alphas = [.0001, .001, .01, .1, 1., 10., 100.]
for a in alphas:
    classifier = SGDClassifier(loss='log', alpha=a, max_iter=1000, tol=1.0e-12, random_state=123)
    classifier.fit(X_train, Y_train)
    
    acc_train.append(accuracy_score(Y_train, classifier.predict(X_train)))
    acc_test.append(accuracy_score(Y_test, classifier.predict(X_test)))
    
    # print the predicted probabilities of the first test instance
    print(classifier.predict_proba(X_test)[0])

plt.semilogx(alphas, acc_train, label='Training')
plt.semilogx(alphas, acc_test, label='Test')

plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.legend(loc='upper right')

plt.show()

In [ ]:
classifier = SGDClassifier(loss='log', max_iter=1000, tol=1.0e-12, random_state=123)
classifier.fit(X_train, Y_train)

taus = [.5, .6, .7, .8, .9, .95, .99]

precs = []
for t in taus:
    precs.append(precision_score(Y_test, threshold(classifier.predict_proba(X_test), t), pos_label="Android"))

plt.plot(taus, precs)

plt.xlabel('Tau')
plt.ylabel('Precision')

plt.show()

In [ ]:
alt.Chart(data).mark_point().encode(
    x = 'FIRE_YEAR',
    y = 'FIRE_SIZE',
    color = 'NWCG_REPORTING_UNIT_ID'
).interactive()

In [ ]:
import matplotlib.pyplot as plt

acc_train = []
acc_test = []

alphas = []
for a in alphas:
    classifier = SGDClassifier(loss='log', alpha=a, max_iter=1000, tol=1.0e-12, random_state=123)
    classifier.fit(X_train, Y_train)
    
    acc_train.append(accuracy_score(Y_train, classifier.predict(X_train)))
    acc_test.append(accuracy_score(Y_test, classifier.predict(X_test)))
    
    # print the predicted probabilities of the first test instance
    print(classifier.predict_proba(X_test)[0])

plt.semilogx(alphas, acc_train, label='Training')
plt.semilogx(alphas, acc_test, label='Test')

plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.legend(loc='upper right')

plt.show()